In [2]:
#import liberaries
import time
from datetime import timedelta, datetime
from pubnub.callbacks import SubscribeCallback
from pubnub.enums import PNStatusCategory, PNOperationType

In [ ]:
# initialize global variables
get_hour = None
get_date = None

set_minutes = 1
set_top_trends = None

creation_time = None
start_time = None
end_time = None
session = None

tweet_count = dict()
trends_tweets = dict()

In [ ]:
def my_publish_callback(envelope, status):
    # Check whether request successfully completed or not
    if not status.is_error():
        pass  # Message successfully published to specified channel.
    else:
        pass  # Handle message publish error. Check 'category' property to find out possible issue
        # because of which request did fail.
        # Request can be resent using: [status retry];

In [ ]:
class FirstSubscribeCallback(SubscribeCallback):
    
    def presence(self, pubnub, presence):
        pass  # handle incoming presence data
    

    def status(self, pubnub, status):
        if status.category == PNStatusCategory.PNUnexpectedDisconnectCategory:
            pass  # This event happens when radio / connectivity is lost

        elif status.category == PNStatusCategory.PNConnectedCategory:
            # Connect event. You can do stuff like publish, and know you'll get it.
            # Or just use the connected event to confirm you are subscribed for
            # UI / internal notifications, etc
            pubnub.publish().channel('pubnub-twitter').message("Hello").pn_async(my_publish_callback)

        elif status.category == PNStatusCategory.PNReconnectedCategory:
            pass
            # Happens as part of our regular operation. This event happens when
            # radio / connectivity is lost, then regained.
        elif status.category == PNStatusCategory.PNDecryptionErrorCategory:
            pass
            # Handle message decryption error. Probably client configured to
            # encrypt messages and on live data feed it received plain text.
    
    def message(self, pubnub, message):
        # Handle new message stored in message.message
        creation_time = datetime.strptime(message.message['created_at'], "%a %b %d %H:%M:%S %z %Y")

        global get_hour, get_date
        if get_hour is None:
            get_hour = creation_time.hour
            get_date = creation_time.date()
            print(get_hour, get_date)
            return
            

In [ ]:
class MainSubscribeCallback(SubscribeCallback):
    
    def status(self, pubnub, status):
         # handle according to status
        FirstSubscribeCallback.status(self, pubnub, status)
        
    def presence(self, pubnub, status):
         # handle incoming presence data
        FirstSubscribeCallback.presence(self, pubnub, presence)
        
    def message(self, pubnub, message):
        # Handle new message stored in message.message
        
        global creation_time, end_time, start_time, tweet_count, set_top_trends, zip_path, cassandra_configs, trends_tweets, session
        
        creation_time = datetime.strptime(message.message['created_at'], "%a %b %d %H:%M:%S %z %Y")
                
        if start_time:
            if creation_time > end_time:
                print(set_minutes, " minutes have passed ", creation_time)
                
                unix_time = int(time.mktime(start_time.timetuple())) 
                key =  str(unix_time) + "_" + str(set_minutes)

                value = dict()
                for trendd, tweet_numbers in tweet_count.items():
                    value[trendd] = tweet_numbers
                    trends_tweets[key] = value
    
                print("Twitter tweets", trends_tweets)
                
                start_time = creation_time
                end_time = creation_time + timedelta(minutes=set_minutes) 
                trends_tweets = dict()
            
        else:
            start_time = creation_time
            end_time = creation_time + timedelta(minutes=set_minutes) 
            print("Process Started at: ", creation_time)
            for trend_number, set_trend in set_top_trends.items():
                tweet_count[set_trend] = 0
                
        trend_list = message.message['entities']['hashtags']
        

        # and set_country in country
        if trend_list:
            for trend in trend_list:
                # print(creation_time, trend, country)
                trend_text = trend['text'].lower()
                for trend_number, set_trend in set_top_trends.items():
                    if set_trend == trend_text:
                        print(set_trend, trend_text)
                        tweet_count[set_trend] +=1